# Visualize precomputed explanations

In this notebook we load explanations that were computed through `explain_all.py` and visualize some of them and do some aggregations.

#### 1. Imports and paths

In [1]:
import json
from pathlib import Path

import numpy as np
import pandas as pd
from dianna import visualization
from dianna.utils.tokenizers import SpacyTokenizer

from train_inlegalbert_xgboost import class_names

In [2]:
model_path = Path('..\inlegal_xgboost_classifier_xgboost_classifier.json')

#### Some test data

In [3]:
constitutive_statement_0 = "The purchase, import or transport from Syria of crude oil and petroleum products shall be prohibited."
constitutive_statement_1 = "This Decision shall enter into force on the twentieth day following that of its publication in the Official Journal of the European Union."
regulatory_statement_0 = "Where observations are submitted, or where substantial new evidence is presented, the Council shall review its decision and inform the person or entity concerned accordingly."
regulatory_statement_1 = "The relevant Member State shall inform the other Member States of any authorisation granted under this Article."
regulatory_statement_2 = "Member States shall cooperate, in accordance with their national legislation, with inspections and disposals undertaken pursuant to paragraphs 1 and 2."

Loading the model



In [4]:
from classify_text_with_inlegal_bert_xgboost import classify_texts


class StatementClassifier:
    def __init__(self):
        self.tokenizer = SpacyTokenizer(name='en_core_web_sm')

    def __call__(self, sentences):
        # ensure the input has a batch axis
        if isinstance(sentences, str):
            sentences = [sentences]

        probs = classify_texts(sentences, model_path, return_proba=True)

        return np.transpose([(probs[:, 0]), (1 - probs[:, 0])])
            

In [5]:
# define model runner. max_filter_size is a property of the model
model_runner = StatementClassifier()

#### Test the model

In [6]:
prediction = model_runner([constitutive_statement_0,constitutive_statement_1, regulatory_statement_0, regulatory_statement_1,regulatory_statement_2])
[class_names[m] for m in np.argmax(prediction, axis=1)]

Some weights of the model checkpoint at law-ai/InLegalBERT were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Creating features: 100%|██████████| 5/5 [00:00<00:00, 17.51it/s]


Loading model from ..\inlegal_xgboost_classifier_xgboost_classifier.json.


['constitutive', 'constitutive', 'regulatory', 'regulatory', 'regulatory']

#### Load precomputed data

In [7]:
p = Path('./results_1000.json')
with open(p, 'r') as f:
    results = json.load(f)

tokenizer = SpacyTokenizer(name='en_core_web_sm')

### Investigate a single statement and its explanation

In [8]:
statement_no = 0
statement = list(results.keys())[statement_no]
print(statement)
prediction = model_runner([statement])[0]
explanation_relevance = results[statement]
print(pd.DataFrame([prediction,], columns=class_names))
top_class = np.argmax(prediction)
print(f'class label *{class_names[top_class]}* was predicted for this sentence')
label_of_interest = top_class
print(pd.DataFrame(explanation_relevance[0], columns=['token','start','weight']).drop(['start'], axis=1))
visualization.highlight_text(explanation_relevance[label_of_interest], tokenizer.tokenize(statement))

This Decision shall apply until 13 April 2014.


Some weights of the model checkpoint at law-ai/InLegalBERT were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Creating features: 100%|██████████| 1/1 [00:00<00:00, 22.16it/s]

   constitutive  regulatory
0      0.998768    0.001232
class label *constitutive* was predicted for this sentence
      token    weight
0     until  0.009208
1      This  0.007477
2  Decision  0.005172
3     shall  0.003755
4         .  0.003539
5     April  0.003357
6        13  0.003058
7     apply  0.002170
8      2014  0.001257


### Explanations of a batch of statements

In [9]:
statements = list(results.keys())[:15]
predictions = model_runner(statements)

for label_of_interest in range(len(class_names)):
    print(f'Explanations for class {class_names[label_of_interest]}')
    for statement, prediction in zip(statements, predictions):
        if np.argmax(prediction) == label_of_interest:
            visualization.highlight_text(results[statement][label_of_interest], tokenizer.tokenize(statement))

Some weights of the model checkpoint at law-ai/InLegalBERT were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Creating features: 100%|██████████| 15/15 [00:01<00:00, 12.69it/s]

Explanations for class constitutive


Explanations for class regulatory


### Aggregated results

In [10]:
label_of_interest = 0
statement_infos = [statement_info[label_of_interest] for _statement, statement_info in results.items()]
token_infos = pd.DataFrame([    x    for xs in statement_infos    for x in xs], columns=['token', 'start', 'weight']).drop('start', axis=1)

token_infos.groupby('token').mean()






,weight
token,
,0.000606
,0.002207
,0.000330
,0.002904
,0.000334
...,...
‘,0.000274
’,-0.001473
’s,0.003515
